# Examples

Let's start by importing the {py:class}`quatorch.Quaternion` class:

In [1]:
from quatorch import Quaternion

/home/egidio/repos/quatorch/.venv/lib/python3.10/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


There are two main ways to initialize a quaternion: from four scalars or from a tensor that has shape $(..., 4)$. Here are two ways to define a 45 degrees rotation around the X axis:

In [2]:
import torch


# Create a quaternion from four scalars (W, X, Y, Z)
q = Quaternion(0.9239, 0.3827, 0.0, 0.0)

# Or from a tensor of shape (..., 4)
q2 = Quaternion(torch.tensor([0.9239, 0.3827, 0.0, 0.0]))

print(f"{(q == q2) = }")

(q == q2) = tensor([True, True, True, True])


As you can see, both methods create the same quaternion. Also notice that element-wise comparison was performed, as a quaternion is still a subclass of tensor. Let's now inspect quaternion multiplication and power:

In [3]:


# Normalize
q = q.normalize()

print(f"{q**4 = }")
print(f"{q*q*q*q = }")

q**4 = Quaternion([-2.9846e-05,  1.0000e+00,  0.0000e+00,  0.0000e+00])
q*q*q*q = Quaternion([-2.9832e-05,  1.0000e+00,  0.0000e+00,  0.0000e+00])


As expected both returned the same result, which represents a 180 degrees rotation around the X axis. We can verify that by converting it to a axis-angle representation:

In [4]:
axis, angle = (q**4).to_axis_angle()
print(f"Axis: {axis}, Angle (radians): {angle.rad2deg()}")


Axis: tensor([1.0000, 0.0000, 0.0000]), Angle (radians): 180.00341796875


Or a rotation matrix:

In [5]:
# Convert to rotation matrix
q.to_rotation_matrix()

tensor([[ 1.0000,  0.0000,  0.0000],
        [ 0.0000,  0.7071, -0.7071],
        [ 0.0000,  0.7071,  0.7071]])

<div class="admonition tip">
<div class="title">Tip</div>
 <p> You may also contruct quaternions from rotation matrix or an axis-angle representation using the methods [`quatorch.Quaternion.from_rotation_matrix`](project:api.md#quatorch.Quaternion.from_rotation_matrix) and [`quatorch.Quaternion.from_axis_angle`](project:api.md#quatorch.Quaternion.from_axis_angle)</p>
</div>


One interesting property of quaternions is that they take value os a 4D sphere. This means that we can interpolate between two quaternions using spherical linear interpolation ([`quatorch.Quaternion.slerp`](project:api.md#quatorch.Quaternion.slerp) ). Let's see how it works:

In [6]:
# Slerp between quaternions
q_final = q**4
t = 1/3
q.slerp(q_final, t)

Quaternion([0.7071, 0.7071, 0.0000, 0.0000])

Notice that this (and all other operations documented in the [API reference](project:api.md) for more details), support broadcasting, so you can work with batches of quaternions seamlessly. In the example above, let's use `t` as a tensor of shape (4, 1) to get a batch of interpolated quaternions:

In [11]:
q.slerp(q_final, torch.linspace(0, 1, steps=4).unsqueeze(-1))  # Batch of interpolation

Quaternion([[ 9.2388e-01,  3.8269e-01,  0.0000e+00,  0.0000e+00],
            [ 7.0710e-01,  7.0712e-01,  0.0000e+00,  0.0000e+00],
            [ 3.8266e-01,  9.2389e-01,  0.0000e+00,  0.0000e+00],
            [-2.9862e-05,  1.0000e+00,  0.0000e+00,  0.0000e+00]])